## Polynomial functions

In [176]:
def getPoly(X, Y, degree):
    '''
    Calculates least squares polynomial fit of 'degree' of the fitting polynomial
    
    Parameters
    ----------
    X : `pd.datetime` array-like
    Y :  array-like of X size
    
    Returns
    -------
    p : `numpy.lib.polynomial.poly1d` object
    '''
    import numpy as np
    import warnings
    warnings.simplefilter('ignore', np.RankWarning)
    import matplotlib.dates as mdates
    
    #convert dates to num values for poly function
    X_num = mdates.date2num(X)
    
    #calculate Polynomial coefficients, highest power first
    #ndarray, shape (deg + 1,) or (deg + 1, K)
    coefs = np.polyfit(X_num, Y, int(degree))

    #Construct the polynomial
    p = np.poly1d(coefs)
    
    return p

def plotPoly(X, Y, p, show=True,x_label=None,y_label=None,title=None,Mtick=True):
    '''
    Creates a Polynomial plot
    
    Parameters
    ----------
    X : `pd.datetime` array-like
    Y :  array-like of X size
    p : `numpy.lib.polynomial.poly1d` object
    show : boolean, display figure at the end of function if True
    xy_label,title : text for labels and plot title
    Mtick : Million tick, if True shows Y ticks in millions (value/1e6)
    
    Returns
    -------
    f : `matplotlib.figure.Figure`
    '''
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    import matplotlib.ticker as ticker

    #buld the plot
    plt.style.use('seaborn-whitegrid')
    f, ax = plt.subplots(figsize=(10,5))
    #plt.style.use('fivethirtyeight')
    #f = plt.figure(figsize=(20,10))
    
    #set y axis scale to million
    if Mtick:
        scale_y = 1e6
        ticks_y = ticker.FuncFormatter(lambda x, pos: '{0:g}'.format(x/scale_y))
        ax.yaxis.set_major_formatter(ticks_y)
        y_label = y_label + ' , Million'
    
    #convert dates to num values for poly function
    X_num = mdates.date2num(X)
    
    plt.plot(X, Y, label='Actual')
    plt.plot(X, p(X_num), "r-", label='Model') #p(X) evaluates the polynomial at X
    
    #ax.set_ylim(0,30*1e6)
    ax.set_xlim(min(X_num),max(X_num))
    
    plt.title(title+' Polynomial Regression', weight='bold')
    plt.ylabel(y_label, weight='bold')
    plt.xlabel(x_label, weight='bold')
    plt.legend()
    
    if show:
        plt.show()
    else:
        plt.close(f)
    return f

def calcPoly(df,degree=3):
    '''
    Returns a DF with calculated polynomial coeffs
    
    Parameters
    ----------
    df : Pandas DataFrame, must have first `Date` column of datetime dtype, 
         other columns should be of `numeric` dtype
    degree : calculate up to degree of power
    '''
    #set column names for the plot excluding 'Date' column [1:]
    columns = df.columns.tolist()[1:]
    
    result_df = pd.DataFrame()
    
    for degree in range(2,degree+1):
        for data in columns:
            temp = df[['Date',data]]
            temp = temp.dropna(how='any')

            #print(f'Getting poly for {data}, {degree}')
            p = getPoly(temp['Date'], temp[data], degree)

            #add coeffs to df
            result_df = pd.concat([result_df,pd.DataFrame(
                {data+'_x_'+str(degree):p.coef[::-1]})],axis=1) 
            #reverse order of poly so column of DF represent power of X

            f = plotPoly(temp['Date'].values, temp[data],p,show=False,x_label='Timeline',y_label=data,
                     title=data+', x'+str(degree),Mtick=False)
            path_to_plot = 'results/plots/'+data+'_polynomial_x'+str(degree)+'.png'
            f.savefig(path_to_plot,dpi=150,transparent=True,bbox_inches='tight') 
    
    return result_df.T

Polynomial with all the metro KPI

In [31]:
#uncomment when will have latest version of the functions
#from polynomial import getPoly
#from polynomial import plotPoly
#from polynomial import calcPoly
import matplotlib.pyplot as plt
import pandas as pd

In [178]:
#calculate metro kpi coeffs
path = 'results/metro_kpi.csv'
metro_kpi = pd.read_csv(path)

metro_kpi['Date'] = pd.to_datetime(metro_kpi['Date'])
metro_kpi = metro_kpi[['Date', 'ROTP', 'RailReliability', 'MetroAccessOTP',
       'EscalatorAvail', 'ElevatorAvail', 'TotalInjuries', 'Crime', 'Metro']]

#convert % to float
metro_kpi['ElevatorAvail'] = [float(x.strip('%'))/100 for x in metro_kpi['ElevatorAvail']]

metro_kpi.head()

,Date,ROTP,RailReliability,MetroAccessOTP,EscalatorAvail,ElevatorAvail,TotalInjuries,Crime,Metro
0,2011-01-01,0.879,48241,0.901,0.888,0.963,2.08,6.39,21082553
1,2011-02-01,0.887,37703,0.890,0.866,0.960,1.66,4.68,21228262
2,2011-03-01,0.910,50328,0.913,0.869,0.969,2.16,3.96,26170157
3,2011-04-01,0.909,39302,0.912,0.862,0.964,2.21,4.72,25656797
4,2011-05-01,0.909,37355,0.922,0.825,0.974,1.69,7.32,24342603


In [179]:
#create all the graphs and combined coeffs table for metro_kpi
kpi = calcPoly(metro_kpi)
kpi.to_csv('results/metro_kpi_poly_coefs.csv')

Polynomial with all combined ridership data

In [181]:
path = 'results/combined.csv'
combined_df = pd.read_csv(path)

combined_df['Date'] = pd.to_datetime(combined_df['Date'])
combined_df.head()

,Date,Bus,Metro,Taxi,Uber
0,2011-01-01,NaN,21082553.0,NaN,NaN
1,2011-02-01,NaN,21228262.0,NaN,NaN
2,2011-03-01,NaN,26170157.0,NaN,NaN
3,2011-04-01,NaN,25656797.0,NaN,NaN
4,2011-05-01,NaN,24342603.0,NaN,NaN


In [182]:
#create all the graphs and combined coeffs table for combined data
combined = calcPoly(combined_df)
combined.to_csv('results/combined_poly_coefs.csv')